In [ ]:
setwd("inputs/crootbox_source")
system("g++ *.cpp -std=c++11 -o crootbox.out")

In [ ]:
########################################################################
# 1 : LOAD THE LIBRARIES AND SOURCE FILES
########################################################################

library(tidyverse)
library(plyr)
library(readr)
library(data.table)
library(dplyr)
library(Matrix)



# Custom functions
source("inputs/io_function.R") # CROOTBOX
source("inputs/getSUF.R") # MARSHAL

# Update the crootbox executable file
# MAC
file.copy("inputs/crootbox_source/crootbox.out", 
          "inputs/crootbox.out", 
          overwrite = T)

In [ ]:
# CROOTBOX PARAMETERS
# B. We load the default parameter sets for the simulation 
rparam <- read_rparam(path = "inputs/param.rparam")
pparam <- read_pparam(path = "inputs/param.pparam")

In [ ]:
# MARSHAL PARAMETERS

# D. We store the input parameters we want to change in vectors, so 
# we can loop on these afterwards
kx_vec <-c(1, 10, 100, 0.1, 0.01)
kr_vec <- c(1, 10, 100, 0.1, 0.01)

# E. We load the default parameter sets for the simulation 
psiCollar <- -15000
soil <- read_csv("inputs/soil.csv")
conductivities <- read_csv("inputs/conductivities.csv")

In [ ]:
########################################################################
# 3 : RUN CROOTBOX
########################################################################

    
    # Modify parmeters
    
    # update "vitesse croissance primaire"
    rparam$val1[rparam$name == "Taproot" & rparam$param == "r"] <- 2 
    
    # update the simulation time
    pparam$val1[pparam$param == "simtime"] <- 15
  
    # update the input text files 
    write_rparam(rparam, "inputs/param.rparam")
    write_pparam(pparam, "inputs/param.pparam")
    
    # Run crootbox
    system("inputs/crootbox.out") # Run crootbox for mac and linux
    # system("inputs/crootbox.exe") # Run crootbox for windows
    
    # Load the simulated data into R to process it and to store it for further use
    current_rootsystem <- fread("outputs/current_rootsystem.txt", header = T)

In [ ]:
# Root system
current_rootsystem %>%
  ggplot() +
  theme_classic() +
  geom_segment(aes(x = x1, y = z1, xend = x2, yend = z2), alpha=0.9) +
  coord_fixed()

In [ ]:
########################################################################
# 4 : RUN MARSHAL
########################################################################
# We loop over the input parameters vectors (see 2.D)

# Load the root system data you would like to use with marshal. 
# It is also possible to loop over a list of root systems is needed
rootsystem <- current_rootsystem
      
# Run MARSHAL
hydraulics <- getSUF(rootsystem, conds, soil, psiCollar)

hydraulic_archi <- hydraulics$root_system
hydraulic_archi$suf <- hydraulics$suf[,1]
hydraulic_archi$kr <- hydraulics$kr[,1]
hydraulic_archi$kx <- hydraulics$kx[,1]
hydraulic_archi$jr <- hydraulics$jr[,1]
hydraulic_archi$jxl <- hydraulics$jxl[,1]
hydraulic_archi$psi <- hydraulics$psi[,1]



# Save all the results of the simulation
save(hydraulic_archi, file = paste0("outputs/marshal/hydraulics_",kx_in,"_",kr_in,".RData"))